<a href="https://colab.research.google.com/github/Contato-Goes/ML_Arvore_Decisao/blob/main/Arvore_decisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>